In [58]:
import pandas as pd

In [59]:
df = pd.read_csv("automation-protocol.csv")

In [60]:
df.columns = df.columns.str.lower()
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20T21:58:04.000Z
1,271,65538,2,1605868423,1,efd275,48,2020-11-20T21:58:04.000Z
2,272,65540,2,1605882795,0,efd275,100,2020-11-20T21:58:04.000Z
3,273,65545,2,1605831076,0,efd275,50,2020-11-20T21:58:04.000Z
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20T21:58:04.000Z


## Data cleaning

In [61]:
df.protocoltime = pd.to_datetime(df.protocoltime)
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 21:58:04+00:00
1,271,65538,2,1605868423,1,efd275,48,2020-11-20 21:58:04+00:00
2,272,65540,2,1605882795,0,efd275,100,2020-11-20 21:58:04+00:00
3,273,65545,2,1605831076,0,efd275,50,2020-11-20 21:58:04+00:00
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20 21:58:04+00:00


In [62]:
df.protocoltime = df.protocoltime.round('15min')
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00+00:00
1,271,65538,2,1605868423,1,efd275,48,2020-11-20 22:00:00+00:00
2,272,65540,2,1605882795,0,efd275,100,2020-11-20 22:00:00+00:00
3,273,65545,2,1605831076,0,efd275,50,2020-11-20 22:00:00+00:00
4,274,65546,2,1605892658,0,f1e0b5,50,2020-11-20 22:00:00+00:00


### Data enriching

In [63]:
devices = df.instanceid.unique()
devices

array([65537, 65538, 65540, 65545, 65546, 65547, 65548, 65549, 65551,
       65552, 65553, 65554, 65555, 65556, 65557])

In [64]:
absolute_min_date = df.protocoltime.min(axis=0)
absolute_max_date = df.protocoltime.max(axis=0)
print("absolute_min_date=%s, absolute_max_date=%s" % (absolute_min_date, absolute_max_date))

complete_time_data = pd.date_range(absolute_min_date, absolute_max_date, freq="15min")
complete_time_data = complete_time_data.tz_localize(None)

complete_arr = []

for device in devices:
    single_device = df[df["instanceid"] == device]
    single_device["protocoltime"] = pd.to_datetime(single_device["protocoltime"]).apply(lambda x: x.replace(tzinfo=None))
    last_state = single_device.iloc[0]
    last_state["onoff"] = 0
    for time_data in complete_time_data.values:
        if time_data in single_device.protocoltime.values:
            # TODO edge case for multiple events in one time slot
            device_row = single_device[single_device["protocoltime"] == time_data]
            complete_arr.append(device_row.values[0])
            last_state = device_row.iloc[0]
        else:
            current_state = last_state.copy()
            current_state["protocoltime"] = time_data
            complete_arr.append(current_state.values)

df = pd.DataFrame(complete_arr, columns=df.columns)
df = df.sort_values(by=["instanceid", "protocoltime"])
df.head()

absolute_min_date=2020-11-20 22:00:00+00:00, absolute_max_date=2020-11-21 20:30:00+00:00


,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00
1,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:15:00
2,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:30:00
3,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:45:00
4,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:00:00


In [65]:
df["weekday"] = df.protocoltime.dt.day_name()
df["weekofyear"] = df.protocoltime.dt.weekofyear
df["year"] = df.protocoltime.dt.year
df["month"] = df.protocoltime.dt.month
df["day"] = df.protocoltime.dt.day
df["time"] = df.protocoltime.dt.strftime("%H:%M")
df.head()

,id,instanceid,type,lastseen,onoff,color,dimmer,protocoltime,weekday,weekofyear,year,month,day,time
0,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:00:00,Friday,47,2020,11,20,22:00
1,270,65537,2,1605867202,1,efd275,51,2020-11-20 22:15:00,Friday,47,2020,11,20,22:15
2,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:30:00,Friday,47,2020,11,20,22:30
3,285,65537,2,1605867202,1,efd275,51,2020-11-20 22:45:00,Friday,47,2020,11,20,22:45
4,304,65537,2,1605867202,0,f1e0b5,51,2020-11-20 23:00:00,Friday,47,2020,11,20,23:00


In [66]:
df.columns

Index(['id', 'instanceid', 'type', 'lastseen', 'onoff', 'color', 'dimmer',
       'protocoltime', 'weekday', 'weekofyear', 'year', 'month', 'day',
       'time'],
      dtype='object')

In [67]:
if "id" in df.columns:
    del df["id"]
if "lastseen" in df.columns:
    del df["lastseen"]
if "protocoltime" in df.columns:
    del df["protocoltime"]
df.head()

,instanceid,type,onoff,color,dimmer,weekday,weekofyear,year,month,day,time
0,65537,2,1,efd275,51,Friday,47,2020,11,20,22:00
1,65537,2,1,efd275,51,Friday,47,2020,11,20,22:15
2,65537,2,1,efd275,51,Friday,47,2020,11,20,22:30
3,65537,2,1,efd275,51,Friday,47,2020,11,20,22:45
4,65537,2,0,f1e0b5,51,Friday,47,2020,11,20,23:00


In [68]:
df.describe()

,instanceid,type,onoff,dimmer,weekofyear,year,month,day
count,1365.000000,1365.0,1365.000000,1365.000000,1365.0,1365.0,1365.0,1365.000000
mean,65548.533333,2.0,0.259341,57.926740,47.0,2020.0,11.0,20.912088
std,6.197609,0.0,0.438433,27.329352,0.0,0.0,0.0,0.283271
min,65537.000000,2.0,0.000000,8.000000,47.0,2020.0,11.0,20.000000
25%,65545.000000,2.0,0.000000,50.000000,47.0,2020.0,11.0,21.000000
50%,65549.000000,2.0,0.000000,50.000000,47.0,2020.0,11.0,21.000000
75%,65554.000000,2.0,1.000000,100.000000,47.0,2020.0,11.0,21.000000
max,65557.000000,2.0,1.000000,100.000000,47.0,2020.0,11.0,21.000000


In [69]:
df.isnull().sum()

instanceid    0
type          0
onoff         0
color         0
dimmer        0
weekday       0
weekofyear    0
year          0
month         0
day           0
time          0
dtype: int64

In [70]:
df.color.unique()

array(['efd275', 'f1e0b5', 'f5faf6', '0'], dtype=object)

In [71]:
df.time.unique()

array(['22:00', '22:15', '22:30', '22:45', '23:00', '23:15', '23:30',
       '23:45', '00:00', '00:15', '00:30', '00:45', '01:00', '01:15',
       '01:30', '01:45', '02:00', '02:15', '02:30', '02:45', '03:00',
       '03:15', '03:30', '03:45', '04:00', '04:15', '04:30', '04:45',
       '05:00', '05:15', '05:30', '05:45', '06:00', '06:15', '06:30',
       '06:45', '07:00', '07:15', '07:30', '07:45', '08:00', '08:15',
       '08:30', '08:45', '09:00', '09:15', '09:30', '09:45', '10:00',
       '10:15', '10:30', '10:45', '11:00', '11:15', '11:30', '11:45',
       '12:00', '12:15', '12:30', '12:45', '13:00', '13:15', '13:30',
       '13:45', '14:00', '14:15', '14:30', '14:45', '15:00', '15:15',
       '15:30', '15:45', '16:00', '16:15', '16:30', '16:45', '17:00',
       '17:15', '17:30', '17:45', '18:00', '18:15', '18:30', '18:45',
       '19:00', '19:15', '19:30', '19:45', '20:00', '20:15', '20:30'],
      dtype=object)

## Data preparation

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=11)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [74]:
y_train_onoff = (df_train.onoff == 1).values
y_val_onoff = (df_val.onoff == 1).values
y_test_onoff = (df_test.onoff == 1).values

In [75]:
del df_train["onoff"]
del df_val["onoff"]
del df_test["onoff"]

### One-Hot encoding

In [76]:
from sklearn.feature_extraction import DictVectorizer

In [77]:
dv = DictVectorizer(sparse=False)

In [78]:
dict_train_onoff = df_train.to_dict(orient="records")

In [79]:
X_train_onoff = dv.fit_transform(dict_train_onoff)

In [80]:
len(dv.vocabulary_)

104